카카오브레인에서 개발한 Pororo(Platform Of neuRal mOdels for natuRal language prOcessing를 활용한 자연어 분석 

https://github.com/kakaobrain/pororo

In [1]:
!pip install pororo

In [2]:
from pororo import Pororo

In [3]:
Pororo.available_tasks()

"Available tasks are ['mrc', 'rc', 'qa', 'question_answering', 'machine_reading_comprehension', 'reading_comprehension', 'sentiment', 'sentiment_analysis', 'nli', 'natural_language_inference', 'inference', 'fill', 'fill_in_blank', 'fib', 'para', 'pi', 'cse', 'contextual_subword_embedding', 'similarity', 'sts', 'semantic_textual_similarity', 'sentence_similarity', 'sentvec', 'sentence_embedding', 'sentence_vector', 'se', 'inflection', 'morphological_inflection', 'g2p', 'grapheme_to_phoneme', 'grapheme_to_phoneme_conversion', 'w2v', 'wordvec', 'word2vec', 'word_vector', 'word_embedding', 'tokenize', 'tokenise', 'tokenization', 'tokenisation', 'tok', 'segmentation', 'seg', 'mt', 'machine_translation', 'translation', 'pos', 'tag', 'pos_tagging', 'tagging', 'const', 'constituency', 'constituency_parsing', 'cp', 'pg', 'collocation', 'collocate', 'col', 'word_translation', 'wt', 'summarization', 'summarisation', 'text_summarization', 'text_summarisation', 'summary', 'gec', 'review', 'review_s

https://github.com/kakaobrain/pororo/tree/master/examples

In [53]:
from pororo import Pororo
import pandas as pd
from sentence_transformers import util
import torch
import numpy as np
import urllib.request

In [54]:
from tqdm import tqdm
tqdm.pandas()

In [55]:
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
Chatbot_Data = pd.read_csv("ChatBotData.csv")

Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [56]:
sTe = Pororo(task="sentence_embedding", lang="ko")
Chatbot_Data['EmbVector'] = Chatbot_Data['Q'].progress_map(lambda x : sTe(x))




100%|██████████| 11823/11823 [04:46<00:00, 41.25it/s]


In [57]:
Chatbot_Data.head()

,Q,A,label,EmbVector
0,12시 땡!,하루가 또 가네요.,0,"[-0.12592222, -0.15255809, -0.7095082, 0.14208..."
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[-0.106658615, 0.022615707, 0.07773947, 0.3133..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[-0.041968275, -0.14296974, 0.08135133, 0.0219..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[-0.024401283, -0.17083658, 0.10665271, -0.024..."
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[0.17281015, -0.21295476, 0.1486542, -0.033685..."


In [58]:
EmbData = torch.tensor(Chatbot_Data['EmbVector'].tolist())

In [59]:
def chat(sent="0"):
  while 1:
    q = input("user > ").strip()
    if q == "quit":
      break
    a = ""
    # Pororo Sentense Embedding으로 텍스트 유사도를 구합니다.
    q = sTe(q)
    # 질문을 Tensor로 바꿉니다.
    q = torch.tensor(q)
    # 코사인 유사도 
    cos_sim = util.pytorch_cos_sim(q, EmbData)

    #유사도가 가장 비슷한 질문 인덱스를 구합니다.
    best_sim_idx = int(np.argmax(cos_sim))

    # 질문의 유사도와 가장 비슷한  답변 제공
    answer = Chatbot_Data['A'][best_sim_idx]
    print(answer)

In [ ]:
chat()

user > 그녀에게 고백하는게 맞을까?
당신의 마음이 정리되었다면 고백해보세요.
user > 데이트 장소 추천좀
영화관 공원 놀이동산 바다 산 축제 도서관 엄청 많죠.
user > 첫 데이트라면?
야경이 멋져요. 야경구경 가세요.
user > 결혼하자고 말할래
해봐요.
user > 그녀가 나랑 결혼해줄까?
능력이 있으면 하면 되죠.
user > 결혼하고싶어
좋은 사람이랑 결혼할 수 있을 거예요.
user > 그녀는 좋은사람이야
유유상종이네요.
user > 여자친구랑 헤어지고싶어
인연이 거기까지 였나봐요.
